In [2]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [3]:
import zipfile

with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("resumes")

print("Extraction Complete ✅")


Extraction Complete ✅


In [4]:
!pip install PyPDF2
!pip install sentence-transformers
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.3 MB/s eta 0:00:00


In [5]:
import os
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
import os
import PyPDF2

resume_texts = []
resume_names = []
resume_categories = []

base_path = "resumes"
count = 0
max_resumes = 2500   #We limit to 200

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".pdf") and count < max_resumes:

            file_path = os.path.join(root, file)
            category = os.path.basename(root)

            try:
                with open(file_path, 'rb') as f:
                    reader = PyPDF2.PdfReader(f)

                    text = ""
                    for page in reader.pages:
                        extracted = page.extract_text()
                        if extracted:
                            text += extracted

                    # Only save if text is meaningful
                    if len(text.strip()) > 100:
                        resume_texts.append(text)
                        resume_names.append(file)
                        resume_categories.append(category)
                        count += 1

                        # Print progress every 20 resumes
                        if count % 20 == 0:
                            print(f"{count} resumes processed...")

            except Exception as e:
                print("Error reading:", file)

print("Total resumes extracted:", len(resume_texts))


20 resumes processed...
40 resumes processed...
60 resumes processed...
80 resumes processed...
100 resumes processed...
120 resumes processed...
140 resumes processed...
160 resumes processed...
180 resumes processed...
200 resumes processed...
220 resumes processed...
240 resumes processed...
260 resumes processed...
280 resumes processed...
300 resumes processed...
320 resumes processed...
340 resumes processed...
360 resumes processed...
380 resumes processed...
400 resumes processed...
420 resumes processed...
440 resumes processed...
460 resumes processed...
480 resumes processed...
500 resumes processed...
520 resumes processed...
540 resumes processed...
560 resumes processed...
580 resumes processed...
600 resumes processed...
620 resumes processed...
640 resumes processed...
660 resumes processed...
680 resumes processed...
700 resumes processed...
720 resumes processed...
740 resumes processed...
760 resumes processed...
780 resumes processed...
800 resumes processed...
820 

In [11]:
print(len(resume_texts))
print(resume_texts[0][:15000])


2483
CASE MANAGER/OUTREACH ADVOCATE
Summary
Adaptable and friendly Case Worker and team player with comprehensive background in crisis intervention utilizing motivational interviewing
techniques.
Highlights
Microsoft Office, Word, Excel, Outlook, Data Entry
and Power Point.
Ability to interact with a diverse population with a compassionate
demeanor.Excellent interpersonal skills
Strong communicator
Culturally-sensitive
Self-starter
Skilled multi-tasker
Fast learner
Detail-oriented
Accomplishments
Spearheaded cell phone collection program for survivors.
Received Employee of the Month on multiple occasions.
Experience
Case Manager/Outreach Advocate
 
June 2008
 
to 
October 2014
 
Company Name
 
ï¼​ 
City
 
, 
State
Caseworker/Outreach Advocate Use Microsoft Word in culmination with Excel to create monthly board reports Responsible for entering
daily client data in the Integrated Tracking System Maintaining client files Responsible for maintaining client confidentiality Answer hotline
ca

In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model Loaded ")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model Loaded 


In [13]:
job_description = """
Looking for a Data Scientist skilled in Python,
Ms Word , SQL, Excel and Data Analysis.
"""


In [14]:
job_embedding = model.encode([job_description])
resume_embeddings = model.encode(resume_texts)


In [15]:
scores = cosine_similarity(job_embedding, resume_embeddings)[0]


In [48]:
results = []

for i in range(len(resume_names)):
    results.append({
        "Name": resume_names[i],
        "Category": resume_categories[i],
        "Match Score": scores[i] * 100
    })

results = sorted(results, key=lambda x: x["Match Score"], reverse=True)

for rank, item in enumerate(results, start=1):
    print(f"Rank {rank}")
    print("Name:", item["Name"])

    print("Match Score: {:.2f}%".format(item["Match Score"]))
    print("-" * 40)


Rank 1
Name: 37242217.pdf
Match Score: 48.63%
----------------------------------------
Rank 2
Name: 20674668.pdf
Match Score: 47.45%
----------------------------------------
Rank 3
Name: 26585242.pdf
Match Score: 46.93%
----------------------------------------
Rank 4
Name: 18448085.pdf
Match Score: 46.58%
----------------------------------------
Rank 5
Name: 29196643.pdf
Match Score: 45.92%
----------------------------------------
Rank 6
Name: 62994611.pdf
Match Score: 45.80%
----------------------------------------
Rank 7
Name: 14871762.pdf
Match Score: 45.80%
----------------------------------------
Rank 8
Name: 16186411.pdf
Match Score: 45.16%
----------------------------------------
Rank 9
Name: 10562768.pdf
Match Score: 44.93%
----------------------------------------
Rank 10
Name: 37764298.pdf
Match Score: 44.49%
----------------------------------------
Rank 11
Name: 26746496.pdf
Match Score: 44.41%
----------------------------------------
Rank 12
Name: 27606527.pdf
Match Score: 4

In [17]:
skills_list = [
    "python", "machine learning", "deep learning", "sql",
    "data analysis", "nlp", "tensorflow", "pytorch",
    "tableau", "excel", "power bi", "java", "c++"
]

def extract_skills(text):
    found_skills = []
    text_lower = text.lower()

    for skill in skills_list:
        if skill in text_lower:
            found_skills.append(skill)

    return found_skills


In [18]:
resume_skills = [extract_skills(text) for text in resume_texts]


In [36]:
job_skills = extract_skills(job_description)

skill_scores = []

for skills in resume_skills:
    match = len(set(skills) & set(job_skills))
    skill_score = (match / len(job_skills)) * 100 if len(job_skills) > 0 else 0
    skill_scores.append(skill_score)


In [37]:
final_scores = []

for i in range(len(scores)):
    combined = (scores[i] * 100 * 0.7) + (skill_scores[i] * 0.3)
    final_scores.append(combined)

In [49]:
results = []

for i in range(len(resume_names)):
    results.append({
        "Name": resume_names[i],
        "Category": resume_categories[i],
        "Similarity Score": scores[i] * 100,
        "Skill Score": skill_scores[i],
        "Final Score": final_scores[i],
        "Skills Found": resume_skills[i]
    })

results = sorted(results, key=lambda x: x["Final Score"], reverse=True)

top_200 = results[:200]

for rank, item in enumerate(top_200, start=1):
    print(f" Rank {rank}")
    print("Name:", item["Name"])

    print("Final Score: {:.2f}%".format(item["Final Score"]))
    print("Skills Found:", item["Skills Found"])
    print("-" * 50)


 Rank 1
Name: 18448085.pdf
Final Score: 62.61%
Skills Found: ['python', 'sql', 'data analysis', 'tableau', 'excel']
--------------------------------------------------
 Rank 2
Name: 12011623.pdf
Final Score: 59.67%
Skills Found: ['python', 'machine learning', 'sql', 'data analysis', 'tableau', 'excel']
--------------------------------------------------
 Rank 3
Name: 22946204.pdf
Final Score: 58.52%
Skills Found: ['python', 'machine learning', 'sql', 'data analysis', 'excel', 'java']
--------------------------------------------------
 Rank 4
Name: 20674668.pdf
Final Score: 55.72%
Skills Found: ['python', 'sql', 'excel', 'java', 'c++']
--------------------------------------------------
 Rank 5
Name: 26585242.pdf
Final Score: 55.35%
Skills Found: ['sql', 'data analysis', 'tableau', 'excel']
--------------------------------------------------
 Rank 6
Name: 62994611.pdf
Final Score: 54.56%
Skills Found: ['python', 'sql', 'tensorflow', 'tableau', 'excel', 'java']
------------------------------

In [50]:
import pandas as pd

df_results = pd.DataFrame(results)
df_results.to_csv("Ranked_Resumes.csv", index=False)

print("CSV Exported Successfully ")


CSV Exported Successfully 


In [51]:
job_skills = extract_skills(job_description)

print("Required Skills:", job_skills)


Required Skills: ['python', 'sql', 'data analysis', 'excel']


In [52]:
missing_skills_list = []

for skills in resume_skills:
    missing = list(set(job_skills) - set(skills))
    missing_skills_list.append(missing)


In [54]:
results = []

for i in range(len(resume_names)):
    results.append({
        "Name": resume_names[i],

        "Similarity Score": scores[i] * 100,
        "Skill Score": skill_scores[i],
        "Final Score": final_scores[i],
        "Skills Found": resume_skills[i],
        "Missing Skills": missing_skills_list[i]
    })


In [56]:
results = sorted(results, key=lambda x: x["Final Score"], reverse=True)

top_200 = results[:200]

for rank, item in enumerate(top_200, start=1):
    print(f" Rank {rank}")
    print("Name:", item["Name"])
    print("Final Score: {:.2f}%".format(item["Final Score"]))
    print("Skills Found:", item["Skills Found"])

    print("-" * 60)


 Rank 1
Name: 18448085.pdf
Final Score: 62.61%
Skills Found: ['python', 'sql', 'data analysis', 'tableau', 'excel']
------------------------------------------------------------
 Rank 2
Name: 12011623.pdf
Final Score: 59.67%
Skills Found: ['python', 'machine learning', 'sql', 'data analysis', 'tableau', 'excel']
------------------------------------------------------------
 Rank 3
Name: 22946204.pdf
Final Score: 58.52%
Skills Found: ['python', 'machine learning', 'sql', 'data analysis', 'excel', 'java']
------------------------------------------------------------
 Rank 4
Name: 20674668.pdf
Final Score: 55.72%
Skills Found: ['python', 'sql', 'excel', 'java', 'c++']
------------------------------------------------------------
 Rank 5
Name: 26585242.pdf
Final Score: 55.35%
Skills Found: ['sql', 'data analysis', 'tableau', 'excel']
------------------------------------------------------------
 Rank 6
Name: 62994611.pdf
Final Score: 54.56%
Skills Found: ['python', 'sql', 'tensorflow', 'tableau

In [57]:
readiness_score = 100 - skill_score


In [58]:
import pandas as pd

df_results = pd.DataFrame(results)

# Sort by Final Score
df_results = df_results.sort_values(by="Final Score", ascending=False)

df_results.head(200)


,Name,Similarity Score,Skill Score,Final Score,Skills Found,Missing Skills
0,18448085.pdf,46.583954,100.0,62.608768,"[python, sql, data analysis, tableau, excel]",[]
1,12011623.pdf,42.392044,100.0,59.674431,"[python, machine learning, sql, data analysis,...",[]
2,22946204.pdf,40.749748,100.0,58.524826,"[python, machine learning, sql, data analysis,...",[]
3,20674668.pdf,47.450211,75.0,55.715145,"[python, sql, excel, java, c++]",[data analysis]
4,26585242.pdf,46.925350,75.0,55.347744,"[sql, data analysis, tableau, excel]",[python]
...,...,...,...,...,...,...
195,88651471.pdf,34.451008,25.0,31.615705,[excel],"[sql, python, data analysis]"
196,25047127.pdf,34.439491,25.0,31.607643,[excel],"[sql, python, data analysis]"
197,87635012.pdf,23.724968,50.0,31.607477,"[sql, excel]","[python, data analysis]"
198,25718772.pdf,34.397301,25.0,31.578110,"[excel, java]","[sql, python, data analysis]"
